# Main Aggregation Script

In [ ]:
#Reloads the lab.py and crystals.py modules to update any changes (after saving)
#If a new method or object is created, autoreload doesn't work and the 
#kernel needs to be closed and halted after saving and making a 'checkpoint'
#in this notebook

%load_ext autoreload
%autoreload 2

In [ ]:
import ipas
import matplotlib.pyplot as plt
import numpy as np
from sqlite3 import Connection as SQLite3Connection
from sqlalchemy import create_engine, event, select
from sqlalchemy.engine import Engine
from sqlalchemy.orm import sessionmaker
import base
from ipas import lab_ice_agg_SQL as lab
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
from dask import delayed
import cloudpickle as pickle
from multiprocessing import Pool
from functools import partial
import logging
import time

In [ ]:
cluster = SLURMCluster(
    queue='kratos',
    walltime='04-23:00:00',
    cores=1,
    memory='7168MiB', #1 GiB = 1,024 MiB
    processes=1)


cluster.scale(20)

In [ ]:
client = Client(cluster)

In [ ]:
client

In [ ]:
phioarr=np.logspace(-2, 2, num=20, dtype=None)#just columns (0,2); plates (-2,0)
#phioarr = phioarr[1:]
lenphio = len(phioarr)

reqarr = [50,60,70,80]
numaspectratios=len(phioarr)
ch_dist='gamma'         #anything other than gamma uses the characteristic from the best distribution pdf (lowest SSE)
nclusters = 300        #changes how many aggregates per aspect ratio to consider
ncrystals = 50       #number of monomers per aggregate 1
minor = 'depth'        #'minorxy' from fit ellipse or 'depth' to mimic IPAS in IDL
rand_orient = True    #randomly orient the seed crystal and new crystal: uses first random orientation
save_plots = False     #saves all histograms to path with # of aggs and minor/depth folder
file_ext = 'eps'

In [ ]:
def main():
 
    '''
    for phio in phioarr:
        for r in reqarr:
            b1 = lab.collect_clusters(phio, r, nclusters=nclusters,
                                ncrystals=ncrystals, rand_orient=rand_orient)     
   
    output = []
    for r in reqarr:
        for phio in phioarr:
    
            print('eq. vol rad', r, phio)
            output.append(delayed(lab.collect_clusters)(phio, r, nclusters=nclusters,
                                                        ncrystals=ncrystals, rand_orient=rand_orient))
    
    print(output)
    print('computing...')
    b1 = client.compute(output) 
    b1 = client.gather(b1)
    '''

    notebook=3
    print(notebook)
    for r in reqarr:
        #b3 = []
        print('r = ',r)
        count = 1

        pool = Pool(processes=20) #pools are reusable
        parallel_clus=partial(lab.collect_clusters, notebook=notebook, r=r, nclusters=nclusters,\
                                ncrystals=ncrystals, \
                                rand_orient=rand_orient)

        start = time.time()
        output = pool.imap_unordered(parallel_clus, phioarr)
        for done in output:
            print("after %3.1fsec: count:%s"%(time.time()-start, count))
            count +=1

#         for res in output:
#             print("(after %3.1fsec)  mono phi:%.3f  count:%s"%(time.time()-start, res[0].mono_phi, count))
#             count += 1
        print('closing')
        pool.close()
        print('joining')
        pool.join()

        
        print('new r')



In [ ]:
if __name__ == '__main__':

    %time main() 
    
#     filename = 'instance_3radii_iceagg_allrand_50xtalstot_20phi_rall_300agg_lastmono'
#     filehandler = open(filename, 'wb')
#     %time pickle.dump(b, filehandler)
#     filehandler.close()
#     print('finished, creating engine 1!')
    
#     engine = create_engine('sqlite:///IPAS_lastmono.sqlite')
#     event.listen(engine, 'connect', _set_sqlite_pragma)
#     base.Base.metadata.create_all(engine, checkfirst=True)
#     Session = sessionmaker(bind=engine)
#     session = Session()
    
#     try:
#         for r in b:
#             for obj in r:
#                 session.add_all(obj)  # crystal id has been appended into cluster relationship
#                 session.commit()

#     except:
#         print('in except')
#         raise
#     session.close() 
#     print('DONE!')
    
    #packed = msgpack.writeResult('msgpack_test', b1)
    #msgpack.unpackb(packed, encoding='utf-8')
    
    #filename = 'instance_3radii_iceagg_allrand_100xtalstot_20phi_r10_1000agg'
    #filehandler = open(filename, 'wb')
    #%time pickle.dump(b1, filehandler)
    #filehandler.close()
    #print('finished!')
    
    #CHANGE LOOKUP TABLE FILES TO HAVE RANGE OF PHI AND R
    #Kratos = 28*4nodes = 112 -> 28 req so 4 aspect ratios at a time
    #would take 5 runs

In [ ]:
def create_engines(eng_name):
    engine=create_engine(eng_name)
    db = 'SELECT * FROM aggregates'
    df = pd.read_sql(db, con=engine)
    return df

# databases = ['sqlite:///IPAS.sqlite', 'sqlite:///IPAS_1.sqlite', 'sqlite:///IPAS_2.sqlite',\
#             'sqlite:///IPAS_3.sqlite', 'sqlite:///IPAS_4.sqlite', 'sqlite:///IPAS_5.sqlite']

databases = ['sqlite:///IPAS.sqlite', 'sqlite:///IPAS_1.sqlite']

pool = Pool(processes=2)
df_all = pool.map(create_engines, databases)
pool.close()

# engine = create_engine('sqlite:///IPAS.sqlite')
# engine1 = create_engine('sqlite:///IPAS_1.sqlite')
# engine2 = create_engine('sqlite:///IPAS_2.sqlite')
# engine3 = create_engine('sqlite:///IPAS_3.sqlite')
# engine4 = create_engine('sqlite:///IPAS_4.sqlite')
# engine5 = create_engine('sqlite:///IPAS_5.sqlite')

# db = 'SELECT * FROM aggregates'
# df = pd.read_sql(db, con=engine)
# df1 = pd.read_sql(db, con=engine1)
# df2 = pd.read_sql(db, con=engine2)
# df3 = pd.read_sql(db, con=engine3)
# df4 = pd.read_sql(db, con=engine4)
# df5 = pd.read_sql(db, con=engine5)


In [ ]:
%time df = dd.read_sql_table('aggregates', 'sqlite:///IPAS.sqlite',\
                       npartitions=10, index_col='id').persist()  

In [ ]:
%time df_1 = dd.read_sql_table('aggregates', 'sqlite:///IPAS_1.sqlite',\
                       npartitions=28, index_col='id').persist()
progress(df_1)

In [ ]:
%time df_2 = dd.read_sql_table('aggregates', 'sqlite:///IPAS_2.sqlite',\
                       npartitions=28, index_col='id')
df_2.persist() 
progress(df_2)

In [ ]:
%time df_3 = dd.read_sql_table('aggregates', 'sqlite:///IPAS_3.sqlite',\
                       npartitions=28, index_col='id')
df_3.persist() 
progress(df_3)

In [ ]:
df_3.persist() 
progress(df_3)

In [ ]:
%time df_4 = dd.read_sql_table('aggregates', 'sqlite:///IPAS_4.sqlite',\
                       npartitions=28, index_col='id')
df_4.persist() 
progress(df_4)

In [ ]:
%time df_5 = dd.read_sql_table('aggregates', 'sqlite:///IPAS_5.sqlite',\
                       npartitions=28, index_col='id')
df_5.persist() 
progress(df_5)

In [ ]:
df_all.memory_usage()

In [ ]:
# engine = create_engine('sqlite:///IPAS_1.sqlite')
# base.Base.metadata.create_all(engine, checkfirst=True)
# Session = sessionmaker(bind=engine)
# session = Session()

engine = create_engine('sqlite:///IPAS_bkup.sqlite', echo=True)
engine.execute("attach database 'IPAS_1.sqlite' as d1;")
engine.execute("attach database 'IPAS_2.sqlite' as d2;")
engine.execute("attach database 'IPAS_3.sqlite' as d3;")
engine.execute("attach database 'IPAS_4.sqlite' as d4;")
engine.execute("attach database 'IPAS_5.sqlite' as d5;")
  
Session = sessionmaker(bind=engine)


session = Session()
inspector = Inspector.from_engine(engine)
print(inspector.get_table_names())


In [ ]:
engines = []
sessions = []
databases=[IPAS.sqlite, IPAS_1.sqlite, IPAS_2.sqlite, IPAS_3.sqlite, IPAS_4.sqlite, IPAS_5.sqlite]
for dbconninfo in databases:
    engine = create_engine(dbconninfo)
    engines.append(engine)
    sessions.append(sessionmaker(bind=engine)())

In [ ]:
print('AGGREGATE PROPERTIES ==============')

#print(session.query(ipas.IceCluster).count())#, crys.IceCrystal)#.filter(crys.IceCrystal.id == 10)

#print(session.query(ipas.IceCluster).filter(ipas.IceCrystal.r == 7).count())
#query = session.query(ipas.IceCluster).filter(ipas.IceCrystal.r == 10).limit(10)

for value in session.query(ipas.IceCrystal.r).distinct():
    print(value)

#for agg in query:
#    print(agg.id, agg.ncrystals, agg.agg_phi, agg.cplx, agg.crystal[0].id,  agg.crystal[0].phi, agg.crystal[0].r)

#print('MONOMER PROPERTIES ================')
#query = session.query(ipas.IceCrystal).all()[0:5]
#for mono in query:
#    print(mono.id, mono.phi, mono.r, mono.rand_orient, mono.aggs.ncrystals, mono.aggs.agg_phi)


In [ ]:
print(b1.rxs.shape)
print(np.max(b1.rxs))
print(np.max(b1.rys))
print(np.max(b1.rzs))

In [ ]:
#a=x b=y c=z

#dunnavan: a>= b >= c
#in our case z > y > x
phiba = b1.rxs/b1.rzs
phica = b1.rys/b1.rzs
#H, xedges, yedges = np.histogram2d(phica[10,:,2],phiba[10,:,2], normed=True)
#plt.imshow(H)
ax = sns.jointplot(x=phiba[1,:,2], y=phica[1,:,2], kind='hex')
ax = sns.jointplot(x=phiba[1,:,2], y=phica[1,:,2], kind='kde')
ax = sns.jointplot(x=phiba[19,:,2], y=phica[19,:,2], kind='hex', color='r')
ax = sns.jointplot(x=phiba[19,:,2], y=phica[19,:,2], kind='kde', color='r')



In [ ]:
n, bins, patches = plt.hist(phiba[10,:,2], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(phica[10,:,2], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)

In [ ]:
#axes rand plates
n, bins, patches = plt.hist(b1.rxs[1,:,2], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rys[1,:,2], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rzs[1,:,2], bins='auto', color='g',
                            alpha=0.7, rwidth=0.85)

In [ ]:
#axes rand columns 
n, bins, patches = plt.hist(b1.rxs[19,:,2], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rys[19,:,2], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rzs[19,:,2], bins='auto', color='g',
                            alpha=0.7, rwidth=0.85)

In [ ]:
n, bins, patches = plt.hist(b1.rxs[1,:,19], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rys[1,:,19], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rzs[1,:,19], bins='auto', color='g',
                            alpha=0.7, rwidth=0.85)

In [ ]:
file = open('instance_3radii_iceagg_allflat_40xtalstot_20phi_r10', 'rb')
b1 = pickle.load(file)

In [ ]:
b5.ch_req_shape = b5.ch_req.reshape(28, 20, 40)[:,:,:-1]
b5.ch_majorax_shape = b5.ch_majorax.reshape(28, 20, 40)[:,:,:-1]
b5.ch_minorax_shape = b5.ch_minorax.reshape(28, 20, 40)[:,:,:-1]
b5.chphi_shape = b5.chphi.reshape(28, 20, 40)[:,:,:-1]
b5.ch_dd_shape = b5.ch_dd.reshape(28, 20, 40)[:,:,:-1] 

In [ ]:
phioarr=np.logspace(-2, 2, num=20, dtype=None)#just columns (0,2); plates (-2,0)
#phioarr = phioarr[1:]
reqarr = [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100,200,300,400,500,600,700,800,900,1000]
ncrystals=39

f1 = open('find_N_lookup_iceagg_flat_rall.dat',"w")
f2 = open('major_axis_lookup_iceagg_flat_rall.dat',"w")
f3 = open('minor_axis_lookup_iceagg_flat_rall.dat',"w")
f4 = open('dd_lookup_iceagg_flat_rall.dat',"w")

for i in range(len(reqarr)):
    for j in range(len(phioarr)):   
        for N in range((ncrystals)):
            #f1.write('%.3f\t %.1f\t %.4f\t %10.4f\t\n'%(b1.phioarr[j], reqarr[i], b1.chphi[N], b1.ch_req[N]))
            f1.write('{:8.4f} {:8.2f} {:8.4f} {:10.2f}\n'.format(b5.phioarr[j], reqarr[i], b5.chphi_shape[i,j,N], b5.ch_req_shape[i,j,N]))
            #f2.write('%.3f\t %.1f\t %.4f\t\n'%(b1.phioarr[j], reqarr[i], b1.ch_majorax[N]))
            f2.write('{:8.4f} {:8.2f} {:10.2f}\n'.format(b5.phioarr[j], reqarr[i], b5.ch_majorax_shape[i,j,N]))
            #print(b1.phioarr[j], reqarr[i], b1.ch_majorax[N])
           
            #minor_axis
            f3.write('{:8.4f} {:8.2f} {:8.4f}\n'.format(b5.phioarr[j], reqarr[i], b5.ch_minorax_shape[i,j,N]))
            #f3.write('%.3f\t %.1f\t %.4f\t\n'%(b1.phioarr[j], reqarr[i], b1.ch_minorax[N]))
            #density change
            f4.write("{:8.4f} {:8.2f} {:8.4f}\n".format(b5.phioarr[j], reqarr[i], b5.ch_dd_shape[i,j,N]))
            #f4.write('%5.3f\t %5.1f\t %.5f\t\n'%(b1.phioarr[j], reqarr[i], b1.ch_dd[N]))
            
f1.close()
f2.close()
f3.close()
f4.close()
